## 1단계: Activation Vector 추출

In [1]:
import os
import numpy as np
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/socar_open_set'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights')
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # 여기서 'cuda'가 출력되어야 GPU와 연결이 됩니다

cuda


In [2]:
# 데이터 전처리 파이프라인에는 이전에 사용했던 create_dataloader를 그대로 사용
def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((320,320), interpolation=nearest_mode),
        torchvision.transforms.CenterCrop((224,224)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((320,320), interpolation=nearest_mode),
        torchvision.transforms.CenterCrop((224,224)),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data

print('슝=3')

슝=3


In [4]:
# 학습 데이터를 연결하고, 미리 준비해 둔 모델을 불러오기
# 클래스가 atower_b5, balsan_b5, balsan_b6, dcube_b6로 총 4개인 모델
train_loader, _train_data = create_dataloader(TRAIN_PATH, 1, False)
target_class_num = len(os.listdir(TRAIN_PATH))

net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

saved_weight_path = os.path.join(MODEL_PATH, 'classifier_acc_0.96008.pth')
net.load_state_dict(torch.load(saved_weight_path, map_location=device))
print('Successfully Loaded the Network Weight!')
net.eval()

net.to(device)

Successfully Loaded the Network Weight!


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

* OpenMax에 필요한 데이터는 분류에 성공한 데이터의 Activation Vector
* Softmax 층에 입력되는 값이 Activation Vector이니 torch.softmax()의 입력이 되는 값에서 뽑아오면 됨

In [5]:
train_preds = list()
train_actvecs = list()
train_outputs_softmax = list()
train_labels = list()

with torch.no_grad():
  for idx, (img, label) in enumerate(train_loader):
      img = img.to(device)
      label = label.to(device)

      out = net(img)
      out_actvec = out.cpu().detach().numpy()[0]
      out_softmax = torch.softmax(out, 1).cpu().detach().numpy()[0]
      out_pred = int(torch.argmax(out).cpu().detach().numpy())
      out_label = int(label.cpu().detach().numpy())

      train_actvecs.append(out_actvec) # component 1: softmax 전의 Activation Vector
      train_preds.append(out_pred) # componenet 2: 각 데이터에 대한 예측값
      train_outputs_softmax.append(out_softmax) # component 3: 각 데이터에 대한 softmax 확률
      train_labels.append(out_label) # component 4: 각 데이터에 대한 Label (정답)

train_actvecs = np.asarray(train_actvecs)
train_preds = np.asarray(train_preds)
train_outputs_softmax = np.asarray(train_outputs_softmax)
train_labels = np.asarray(train_labels)

* 모델에서 나온 Activation Vector를 모두 사용하지 않는다는 점
* OpenMax에서는 모델이 정답을 맞힌 Activation Vector만 사용
* 올바른 경우의 Activation Vector만 사용하겠다는 이야기

In [6]:
train_correct_actvecs = train_actvecs[train_labels==train_preds]
train_correct_labels = train_labels[train_labels==train_preds]
print('Activation vector: ', train_correct_actvecs.shape)
print('Labels: ', train_correct_labels.shape)

Activation vector:  (4790, 4)
Labels:  (4790,)


## 2단계: Weibull-Distribution
* 앞서 추출해 낸 Activation Vector는 아래처럼 사용됨
* 1.Activation Vector를 클래스마다 나눠 담습니다.
* 2.클래스별로 나눠진 Activation Vector별 평균으로부터 가장 먼 100개의 Vector를 이용해 베이불 분포의 모수를 추출합니다.
* 3.각 클래스당 베이불 분포의 모수들을 저장해 둡니다.
* 우리가 사용할 베이불 분포의 모수는 shape, loc, scale로 3개이고, 클래스는 4개이므로 총 12개의 숫자가 있음
* 아직 클래스가 5개가 되지 않았다는 점에 유의

In [8]:
# 베이불 분포의 모수를 얻기
class_means = list()
dist_to_means = list()
mr_models = {}

for class_idx in np.unique(train_labels):
    
    print('class_idx: ', class_idx)
    class_act_vec = train_correct_actvecs[train_correct_labels==class_idx]
    print(class_act_vec.shape)
    
    class_mean = class_act_vec.mean(axis=0)
    class_means.append(class_mean)
    
    dist_to_mean = np.square(class_act_vec - class_mean).sum(axis=1) # 각 activation vector의 거리를 계산
    dist_to_mean_sorted = np.sort(dist_to_mean).astype(np.float64) # 거리를 기준으로 오름차순 정렬
    dist_to_means.append(dist_to_mean_sorted)

    shape, loc, scale = stats.weibull_max.fit(dist_to_mean_sorted[-100:]) # 거리가 가장 먼 100개를 사용하여 모수 추출
    
    mr_models[str(class_idx)] = {
        'shape':shape,
        'loc':loc,
        'scale':scale
    }
    
class_means = np.asarray(class_means)

class_idx:  0
(1250, 4)
class_idx:  1
(1170, 4)
class_idx:  2
(1179, 4)
class_idx:  3
(1191, 4)


↑
* 이렇게 얻은 모수는 shape, loc, scale
* shape은 베이불 분포의 모양을 결정
<img src="https://i.ibb.co/5WDMjV3/1109-1.jpg">
* loc은 분포의 가로축 평행 이동을 뜻하고, scale은 분포가 얼마나 넓게 퍼져있는지는 뜻함
<img src="https://i.ibb.co/GckMK4p/1109-3.jpg">
* 정상적으로 분류된 Activation Vector를 이용해 베이불 모수를 얻었다는 것은 극단값을 판별할 기준을 얻었다는 이야기

## OpenMax 적용
* 앞서 계산된 모수는 극단값을 판별할 기준
* 모든 클래스에서 평범하지 않다고 판단된다면 어떤 클래스에도 속하지 않는다는 이야기이니 reject라는 클래스로 분류
* OpenMax 확률을 계산하는 함수를 만들어 보기

##### OpenMax 확률
* 지금까지 계산한 것은 베이불 분포의 모수 뿐
* 이미지와 베이불 분포의 모수를 이용해서 최종 분류를 할 수 있는 기준을 만들기
* 우선 베이불 분포로부터 이미지의 확률을 계산
* 베이불 분포 확률이 높다는 이야기는 평범하지 않다고 판별하는 것 즉, 확률이 낮을수록 평범하다는 이야기
* weight×ActivationVector
* 각 이미지의 확률(score)을 뺀 값을 가중치(weight)
* 이 가중치를 Activation Vector에 곱해줌
* 만약 각 이미지가 정해진 클래스에 속할수록 이 값은 높은 값이 나옴
* 반대로 어떤 클래스에 속하지 않을수록 낮은 값이 나옴
* 마지막에 reject class를 추가하여 새로운 값을 추가
* ∑(1−weight)(ActivationVector)
* 위 식을 계산하면 reject class에 속할수록 높은 값이 나옴
* 만약 모든 클래스에 포함될 확률이 낮다면 이 값만 높게 나옴
* 이걸 reject확률로 결정

In [9]:
# reject 클래스 추가
def compute_openmax(actvec, class_means, mr_models):
    dist_to_mean = np.square(actvec - class_means).sum(axis=1)

    scores = list()
    for class_idx in range(len(class_means)):
        params = mr_models[str(class_idx)]
        score = stats.weibull_max.cdf(
            dist_to_mean[class_idx],
            params['shape'],
            params['loc'],
            params['scale']
        )
        scores.append(score)
    scores = np.asarray(scores)
    
    weight_on_actvec = 1 - scores # 각 class별 가중치
    rev_actvec = np.concatenate([
        weight_on_actvec * actvec, # known class에 대한 가중치 곱
        [((1-weight_on_actvec) * actvec).sum()] # unknown class에 새로운 계산식
    ])
    
    openmax_prob = np.exp(rev_actvec) / np.exp(rev_actvec).sum()
    return openmax_prob

### Inference 함수
* threshold 기법
* 계산한 최대 확률이 모두 다 낮은 경우라면 강제로 reject클래스로 분류해주는 방법

In [10]:
def inference(actvec, threshold, target_class_num, class_means, mr_models):
    openmax_prob = compute_openmax(actvec, class_means, mr_models)
    openmax_softmax = np.exp(openmax_prob)/sum(np.exp(openmax_prob))

    pred = np.argmax(openmax_softmax)
    if np.max(openmax_softmax) < threshold:
        pred = target_class_num
    return pred

In [11]:
# Threshold 탐색을 쉽게 하기 위해 함수
def inference_dataloader(net, data_loader, threshold, target_class_num, class_means, mr_models, is_reject=False):
    result_preds = list()
    result_labels = list()

    with torch.no_grad():
      for idx, (img, label) in enumerate(data_loader):
          img = img.to(device)
          label = label.to(device)

          out = net(img)
          out_actvec = out.cpu().detach().numpy()[0]
          out_softmax = torch.softmax(out, 1).cpu().detach().numpy()[0]
          out_label = int(label.cpu().detach().numpy())

          pred = inference(out_actvec, threshold, target_class_num, class_means, mr_models)
      
          result_preds.append(pred)
          if is_reject:
              result_labels.append(target_class_num)
          else:
              result_labels.append(out_label)

    return result_preds, result_labels

* 일반 모델이 가진 분류 능력을 최대한 유지하면서 reject에 대한 판별도 높아야 함

In [12]:
# 0.35로 threshold를 정하고 탐색
test_loader, _test_data = create_dataloader(TEST_PATH, 1, False)
reject_loader, _reject_data = create_dataloader(REJECT_PATH, 1, False)
target_class_num = len(os.listdir(TEST_PATH))

test_preds, test_labels = inference_dataloader(net, test_loader, 0.35, target_class_num, class_means, mr_models)
reject_preds, reject_labels = inference_dataloader(net, reject_loader, 0.35, target_class_num, class_means, mr_models, is_reject=True)

print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

Test Accuracy:  0.939
Reject Accuracy:  0.525


In [13]:
# 0.4 threshold를 정하고 탐색
test_preds, test_labels = inference_dataloader(net, test_loader, 0.4, target_class_num, class_means, mr_models)
reject_preds, reject_labels = inference_dataloader(net, reject_loader, 0.4, target_class_num, class_means, mr_models, is_reject=True)

print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

Test Accuracy:  0.608
Reject Accuracy:  0.845


↑
* 오위치 이미지에 대한 성능은 크게 증가했지만, 정위치 이미지에 대한 성능이 너무 하락함

In [14]:
# 0.38 hreshold를 정하고 탐색
test_preds, test_labels = inference_dataloader(net, test_loader, 0.38, target_class_num, class_means, mr_models)
reject_preds, reject_labels = inference_dataloader(net, reject_loader, 0.38, target_class_num, class_means, mr_models, is_reject=True)

print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

Test Accuracy:  0.888
Reject Accuracy:  0.712
